In [1]:
import os
import torch
from datasets import load_dataset, load_metric
from transformers import (
    PreTrainedTokenizerFast,
    Trainer, 
    TrainingArguments, 
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
import sentencepiece as spm

In [2]:
translation_dataset = load_dataset("wmt14", "de-en", split="train[:1%]")
sentiment_dataset = load_dataset("amazon_polarity", split="train[:1%]")  # Just an example in English, replace with multilingual.

In [3]:
#Phase 2: Experiments (Fine-tuning Models)

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM  # For translation tasks, e.g., MarianMT

def load_custom_tokenizer(tokenizer_type="bpe"):
    if tokenizer_type == "bpe":
        # Load a fast tokenizer from files
        # This is a simplified example, in practice you'd wrap it with PreTrainedTokenizerFast
        return AutoTokenizer.from_pretrained("tokenizers/bpe", use_fast=True)
    elif tokenizer_type == "sp":
        return AutoTokenizer.from_pretrained("tokenizers/sp_unigram_hf", use_fast=True)
    elif tokenizer_type == "wp":
        return AutoTokenizer.from_pretrained("tokenizers/wp", use_fast=True)
    else:
        raise ValueError("Unsupported tokenizer type")

baseline_tokenizer = load_custom_tokenizer("sp")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [4]:
split_dataset = sentiment_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

def preprocess_function_sentiment(examples):
    texts = examples["content"]
    labels = examples["label"]
    tokenized_inputs = baseline_tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = train_dataset.map(preprocess_function_sentiment, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function_sentiment, batched=True)

tokenized_train = tokenized_train.remove_columns(["content", "label"])
tokenized_eval = tokenized_eval.remove_columns(["content", "label"])

tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")


Map:   0%|          | 0/28800 [00:00<?, ? examples/s]

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

In [5]:
# For translation tasks, you might use a MarianMT model or mBART, for sentiment XLM-R or mBERT.
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model_name = "bert-base-uncased" 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
train_args = TrainingArguments(
    output_dir="checkpoints/sentiment_sp",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="logs",
    num_train_epochs=10,          
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=(device.type == "cuda"),
    push_to_hub=False,
)

metric_accuracy = load_metric("accuracy")

/Users/sg/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/x8/bnjqxpd164n045xxqsqmvx9w0000gn/T/ipykernel_74383/1736486005.py:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_accuracy = load_metric("accuracy")
/Users/sg/anaconda3/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=Tru

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric_accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=baseline_tokenizer,
    compute_metrics=compute_metrics
)

/var/folders/x8/bnjqxpd164n045xxqsqmvx9w0000gn/T/ipykernel_74383/1230184579.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
trainer.train()

  0%|          | 0/18000 [00:00<?, ?it/s]

{'loss': 0.6997, 'grad_norm': 1.7761789560317993, 'learning_rate': 4.8611111111111115e-05, 'epoch': 0.28}
{'loss': 0.6975, 'grad_norm': 1.7528256177902222, 'learning_rate': 4.722222222222222e-05, 'epoch': 0.56}
{'loss': 0.6958, 'grad_norm': 5.4670729637146, 'learning_rate': 4.5833333333333334e-05, 'epoch': 0.83}


  0%|          | 0/450 [00:00<?, ?it/s]

{'eval_loss': 0.697838544845581, 'eval_accuracy': 0.4915277777777778, 'eval_runtime': 239.2729, 'eval_samples_per_second': 30.091, 'eval_steps_per_second': 1.881, 'epoch': 1.0}
{'loss': 0.6952, 'grad_norm': 1.9082987308502197, 'learning_rate': 4.4444444444444447e-05, 'epoch': 1.11}
{'loss': 0.6981, 'grad_norm': 1.2722952365875244, 'learning_rate': 4.305555555555556e-05, 'epoch': 1.39}
{'loss': 0.6961, 'grad_norm': 2.096461296081543, 'learning_rate': 4.166666666666667e-05, 'epoch': 1.67}
{'loss': 0.696, 'grad_norm': 1.2440341711044312, 'learning_rate': 4.027777777777778e-05, 'epoch': 1.94}


  0%|          | 0/450 [00:00<?, ?it/s]

...

  0%|          | 0/450 [00:00<?, ?it/s]

{'eval_loss': 0.693058967590332, 'eval_accuracy': 0.5084722222222222, 'eval_runtime': 236.6355, 'eval_samples_per_second': 30.427, 'eval_steps_per_second': 1.902, 'epoch': 9.0}
{'loss': 0.694, 'grad_norm': 1.4425933361053467, 'learning_rate': 4.166666666666667e-06, 'epoch': 9.17}
{'loss': 0.6934, 'grad_norm': 2.410526990890503, 'learning_rate': 2.777777777777778e-06, 'epoch': 9.44}
{'loss': 0.6928, 'grad_norm': 1.5421983003616333, 'learning_rate': 1.388888888888889e-06, 'epoch': 9.72}
{'loss': 0.6936, 'grad_norm': 1.473081350326538, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/450 [00:00<?, ?it/s]

{'eval_loss': 0.6930209398269653, 'eval_accuracy': 0.5084722222222222, 'eval_runtime': 118.9082, 'eval_samples_per_second': 60.551, 'eval_steps_per_second': 3.784, 'epoch': 10.0}
{'train_runtime': 33435.6222, 'train_samples_per_second': 8.614, 'train_steps_per_second': 0.538, 'train_loss': 0.6946745435926649, 'epoch': 10.0}


TrainOutput(global_step=18000, training_loss=0.6946745435926649, metrics={'train_runtime': 33435.6222, 'train_samples_per_second': 8.614, 'train_steps_per_second': 0.538, 'total_flos': 1.894399598592e+16, 'train_loss': 0.6946745435926649, 'epoch': 10.0})

In [9]:
results = trainer.evaluate()
print("Evaluation Results:", results)


sample_text = "I absolutely loved this product, it exceeded my expectations!"
encoded = baseline_tokenizer(sample_text, return_tensors="pt").to(device)
with torch.no_grad():
    output = model(**encoded)
    pred = output.logits.argmax(dim=-1).item()
    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"Review: {sample_text}\nPredicted Sentiment: {sentiment}")

  0%|          | 0/450 [00:00<?, ?it/s]

Evaluation Results: {'eval_loss': 0.6930047869682312, 'eval_accuracy': 0.5084722222222222, 'eval_runtime': 180.6422, 'eval_samples_per_second': 39.858, 'eval_steps_per_second': 2.491, 'epoch': 10.0}
Review: I absolutely loved this product, it exceeded my expectations!
Predicted Sentiment: Positive
